# Explaining the games.py module
*Author: Chirag Vartak*<br>
*Date: 12th March 2016*

## An Introduction, and some other (un)essential information

  Hello all!  
  In this IPython notebook, I plan to help you a little so that you will be able to use the [games.py](https://github.com/aimacode/aima-python/blob/master/games.py). You might already know that the `games.py` module implements the algorithms in Chapter 5 (Adversarial Search) of the book 'Artificial Intelligence: A Modern Approach'. The code in this notebook, and the entire [aima-python](https://github.com/aimacode/aima-python) repository is intended to work with Python 3 (specifically, Python 3.5). So if you happen to be working on Python 2, you should switch to Python 3. If you do not have Python 3 installed, you might want to get that done first. Or even better, install [Anaconda](https://www.continuum.io/downloads) and you will get Jupyter Notebook and IPython along with it. This way you will be able to run both Python 2 and Python 3 using what they call 'virtual environments'. This is the way to go if you don't yet want to let go of your dear old Python 2.7. And this is what I do anyways.  
  
  What we will do to learn to use the code in this module is simply dive in! I feel this is the correct approach as I assume you must have already read Chapter 5 of AIMA. If you haven't, you might want to go back and do that first. If you are tired (or just lazy), at least read the chapter upto Sec. 5.3 because this module covers the algorithms only till that sections anyway. So, I will start by explaining what the class `Game` is and then we will immediately start implementing the `TicTacToe` game. After we define the rules of the `TicTacToe` game, we will create AI players who use different search strategies, namely Minimax Search and Alpha-Beta Search. We will make these players play among themselves, and later on we ourselves will play against these AI players (Yay!).  
The reason I choose the `TicTacToe` game for demonstration of this module should be obvious to you. Everyone knows it and has played it, it is analyzed in quite some detail in AIMA, and most importantly, it has comparatively few states (fewer than 362,880) so that we can explore the search tree completely.  
  
  So let's begin.

## Implementing TicTacToe

To use the code in `games.py` let's import everything from it:

In [5]:
from games import *

Note that, here, as the module `games.py` does a `from utils import *`, all the names (global variables, functions etc.) available in `utils.py` are directly available to us now

### The class `Game`  
  
Let's have a look at the class `Game` in our module. We see that it has six functions, namely `actions`, `result`, `utility`, `terminal_test`, `to_move` and `display`. We see that these functions have not actually been implemented. This class is actually just a template class; we are supposed to create the class for our game, `TicTacToe` by inheriting this `Game` class and implement all the methods mentioned in `Game`. If you forget to implement any one of those a `NotImplementedError` will be raised. So, in this sense, the `Game` class is what you might call an abstract class in Java: it implements nothing, just tells you all that you are supposed to implement and screams at you if forget to implement what it asks.  
  
  Now let's get into some details of all these methods in our `Game` class. You have implement these methods when you create the new class that would represent your game.
  
* `__init__(self, <other variables>)` :  When you create a class inherited from the `Game` class, class `TicTacToe` you'll have to create an object of this inherited class to initialize the game. This initialization might require some additional information which would be passed to `__init__` as variables. For the case of our `TicTacToe` game, this additional information would be the number of rows `h`, number of columns `v` and how many consecutive X's or O's are needed in a row, column or diagonal for a win `k`. Also, the initial game state has to be defined here in `__init__`.
* `actions(self, state)` : Given a state, this method should generates all the legal actions possible from this state, as a list or a generator. Returning a generator rather than a list has the advantage that it saves space and you can still operate on it as a list.
* `result(self, state, move)` : Given a game state and a move, this method returns the game state that you get by making that move on this game state.
* `utility(self, state, player)` : Given a terminal game state and a player, this method returns the utility for that player in the given terminal game state. While implementing this method assume that the game state is a terminal game state. The logic in this module is such that this method will be called only on terminal game states.
* `terminal_test(self, state)` : Given a game state, this method should return `True` if this game state is a terminal state, and `False` otherwise.
* `to_move(self, state)` : Given a game state, this method returns the player who is to play next. This information is typically stored in the game state, so all this method does is extract this information and return it.

### Deciding the game state representation
  
  Now, before we start implementing our `TicTacToe` game, we need to decide how we will be representing our game state. Typically, a game state will give you all the current information about the game at any point in time. Yes, all of it. When you are given a game state, you should be able to tell whose turn it is next, how the game will look like on a real-life board (if it has one) etc. A game state need not include the history of the game. If your you can play the game further given a game state, you game state representation is acceptable. While we might like to include all kinds of information into our game state, we wouldn't want to put too much information into it. Modifying this game state to generate a new one would be a real pain then.
  
  Now, as for our `TicTacToe` game state, would storing only the positions of all the X's and O's be sufficient to represent all the game information at that point in time? Well, does it tell us whose turn it is next? Looking at the 'X's and O's on the board and counting them should tell us that. But that would mean extra computing. To avoid this, we will also store whose move it is next in the game state.  
  
  Think about what we've done here. We have reduced extra computation by storing additional information in a game state. Now, this information might not be absolutely essential to tell us about the state of the game, but it does save us additional computation time. We'll do more of this later on.  
  
  The `TicTacToe` game defines its game state as:

In [6]:
GameState = collections.namedtuple('GameState', 'to_move, utility, board, moves')

The game is called, quite appropriately, as `GameState`, and it has 4 variables, namely, `to_move`, `utility`, `board` and `moves`.  
  
  I'll describe these variables in some more detail:
  
* `to_move` : It represents whose turn it is to move next. This will be a string of a single character, either 'X' or 'O'.
* `utility` : It stores the utility of the game state. Storing this utility is a good idea, because, when you do a Minimax Search or an Alphabeta Search, you generate many recursive calls, whyich travel all the way down to the terminal states. When these recursive calls go back up to the original callee, we have calculated utilities for many game states. We store these utilities in their respective `GameState`s to avoid calculating them all over again.
* `board` : A dict that stores all the positions of X's and O's on the board
* `moves` : It stores the list of legal moves possible from the current position. Note here, that storing the moves as a list, as it is done here, increases the space complexity of Minimax Search from `O(m)` to `O(bm)`. Refer to Sec. 5.2.1 of the book.

### Representing a move  
  
  Now that we have decided how our game state will be represented, it's time to decide how our move will be represented. My advice on this: keep it simple. Becomes easy to use this move to modify a current game state to generate a new one.  
  
  For our `TicTacToe` game, we'll just represent a move by a tuple, where the first and the second elements of the tuple will represent the row and column, respectively, where the next X or O is to be made. Whether to make an X or an O will be decided by the `to_move` variable in the `GameState`.

### The class `TicTacToe`  
  
  Take a look at the class `TicTacToe`. All the methods mentioned in the class `Game` have been implemented here. Some points to note in this class might be:  
    
* The class `TicTacToe` has been inherited fro the class `Game`. As I mentioned earlier, you really want to do this. Catching bugs and errors becomes a whole lot easier.
* A `display` function has been implemented. This function prints the given game state on the console. This might come in handy for debugging and is great when we play ourselves against AIs that we will be creating.
* Additional functions `compute_utility` and `k_in_a_row` are created, which are used by other functions. Well, no one said that you can't do this.

## Creating players to play the games  

## `random_player` and `alphabeta_player`  
  
  So, we have finished implementation of the `TicTacToe` class. What this class does is that, it just defines the rules of the game. We need more to create an AI that can actually play the game. This is where `random_player` and `alphabeta_player` come in.  
  
  The `random_player` is a function that plays random moves in the game. That's it. There isn't much more to this guy.  
  
  The `alphabeta_player`, on the other hand, calls the `alphabeta_full_search` function, which returns the best move in the current game state. Thus, the `alphabeta_player` always plays the best move given a game state, assuming that the game tree is small enough to search entirely.

## `query_player` and `play_game`  
  
  The `query_player` function allows you, a human opponent to play the game. This function requires a `display` method to be implemented in your game class, so that successive game states can be displayed on the terminal, making it easier for you to visualize the game and play accordingly.  
  
  The `play_game` function will the one that will be actually used to play the game. You pass as arguments to it, the an instance of the game you want to play and the players you want in this game. Use it to play AI vs AI, AI vs human, or even human vs human matches!

# Examples  
  
  I will be showing some code examples below that you can run. The games classes which I will use are `TicTacToe` and the `Fig52Game`. The `Fig52Game` is already implemented (actually both are) in the module. This is that small game in Fig 5.2 of the book.  
  
  Have fun executing and modifying these examples!